###### import modules

In [20]:
import sys

sys.path.append('/home/baon/nipype')
sys.path.append('/home/baon/PycharmProjects/3_Dicom_handling/')
sys.path

['/usr/lib/python35.zip',
 '/usr/lib/python3.5',
 '/usr/lib/python3.5/plat-x86_64-linux-gnu',
 '/usr/lib/python3.5/lib-dynload',
 '',
 '/home/baon/.local/lib/python3.5/site-packages',
 '/usr/local/lib/python3.5/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.5/dist-packages/IPython/extensions',
 '/home/baon/.ipython',
 '/home/baon/nipype',
 '/home/baon/PycharmProjects/3_Dicom_handling/',
 '/home/baon/nipype',
 '/home/baon/PycharmProjects/3_Dicom_handling/']

In [21]:
from nipype.interfaces.dcm2nii import Dcm2nii

In [22]:
import pandas as pd
import string
import os
import pydicom
import shutil
import matplotlib.pyplot as plt
import pydicom
import time
from IPython.display import clear_output


import string

import numpy as np
import subprocess

import SimpleITK as sitk
from dicom_funcs_cys import *

dicom=pydicom

In [23]:
rootpath = '/media/baon/baondata2/0_dcm_source/'
os.chdir(rootpath)

###### Set transferred date 

In [24]:
date = 20190731

In [25]:
if not os.path.isdir('./0_0_dcm_glioma_original/a_%s/'%date):
    os.mkdir('./0_0_dcm_glioma_original/a_%s/'%date)

##### list_downloaded = []
src_root =  './0_2_dcm_unsorted/%s_original/'%date #### path of unsorted dicom files
for root, folders, files in os.walk(src_root):
    for folder in folders:
        print(folder)
        for _,_,files2 in os.walk(os.path.join(root, folder)):
            #print(files2[0])
            ds2 = dicom.read_file(os.path.join(root,folder,files2[0]))
            print((ds2.PatientID))
            list_downloaded.append(ds2.PatientID)
            
len(list_downloaded)


list_downloaded = [int(x) for x in list_downloaded]
list_downloaded

whole_list0 = pd.read_csv('./0_2_dcm_unsorted/Glioma_newlist_20190803.csv')

whole_list = list(whole_list0['unitno'])
len(whole_list)

add_list = []
for unitno in whole_list:
    if not unitno in list_downloaded:
        add_list.append(unitno)
        
print(add_list)
print(len(add_list))


In [29]:
df_fileno = []

src_root= '/media/baon/baondata2/0_dcm_source/0_2_dcm_unsorted/20190731_original/'
for root, folders, files in os.walk(src_root):
    for folder in folders: 
        #print(os.path.join(root,folder))
        files = [f for f in os.listdir(os.path.join(root, folder)) if os.path.isfile(os.path.join(root,folder,f))]
        
        ds2 = dicom.read_file(os.path.join(root,folder,files[0]))
        df_fileno_sub={'unitno':ds2.PatientID, 'mrdate':ds2.StudyDate  }
        df_fileno.append(df_fileno_sub)


[{'mrdate': '20190213', 'unitno': '8254998'},
 {'mrdate': '20180911', 'unitno': '8725351'},
 {'mrdate': '20180628', 'unitno': '8681862'},
 {'mrdate': '20181023', 'unitno': '8751259'},
 {'mrdate': '20190412', 'unitno': '8852436'},
 {'mrdate': '20181110', 'unitno': '8760750'},
 {'mrdate': '20180913', 'unitno': '5201788'},
 {'mrdate': '20181108', 'unitno': '5005671'},
 {'mrdate': '20181230', 'unitno': '7471985'},
 {'mrdate': '20190220', 'unitno': '8807774'},
 {'mrdate': '20181207', 'unitno': '8778106'},
 {'mrdate': '20190604', 'unitno': '10012748'},
 {'mrdate': '20190116', 'unitno': '8802114'},
 {'mrdate': '20190410', 'unitno': '8855099'},
 {'mrdate': '20180929', 'unitno': '8736479'},
 {'mrdate': '20180728', 'unitno': '2826490'},
 {'mrdate': '20180417', 'unitno': '8591024'},
 {'mrdate': '20190312', 'unitno': '8815190'},
 {'mrdate': '20190507', 'unitno': '10005178'},
 {'mrdate': '20180805', 'unitno': '8700503'},
 {'mrdate': '20190105', 'unitno': '3679945'},
 {'mrdate': '20181230', 'unitno'

In [31]:
df_fileno= pd.DataFrame(df_fileno)
df_fileno.to_csv('glioma_since201806_mrdate.csv')
            

## Sort all dicom files
#### according to untino \ unitno_studydate_studyname \ seqname \ unitno_studydate_seqno_instantceno.dcm


In [7]:
def func_sort_dicoms(src_root, trg_root):

    for root, _, files in os.walk(src_root):
        for file in files :

            src_path = os.path.join(root, file)
            ds2 = dicom.read_file(src_path)
            #print(src_path)
            #print(ds2.PatientID + "_" + ds2.StudyDate)
            
            try: 
                safe_studyseq = safe_naming("%s_%s_%s" % (ds2.PatientID, ds2.StudyDate, ds2.StudyDescription), ds2.SeriesDescription)
                trg_folder = os.path.join(trg_root, ds2.PatientID, safe_studyseq)
            except:
                try:
                    safe_studyseq = safe_naming("%s_%s_%s" % (ds2.PatientID, ds2.StudyDate, ds2.StudyDescription), "no_seriesname")
                    trg_folder = os.path.join(trg_root, ds2.PatientID, safe_studyseq)
                    #print(unitno)
                except: 
                    safe_studyseq = safe_naming("%s_%s_%s" % (ds2.PatientID, ds2.StudyDate, "no_studyname"), ds2.SeriesDescription)
                    trg_folder = os.path.join(trg_root, ds2.PatientID, safe_studyseq)

                
            if not os.path.isdir(trg_folder):
                os.makedirs(trg_folder)
            
            trg_path = os.path.join(trg_folder, '%s_%s_%s_%s.dcm'% (ds2.PatientID, ds2.StudyDate, ds2.SeriesNumber, ds2.InstanceNumber))

            #print(src_path)
            #print(trg_folder)
            #print(trg_path) 
            if not os.path.isfile(trg_path):
                shutil.copy(src_path, trg_path)
                #print("being copied")


In [8]:

src_root =  './0_2_dcm_unsorted/%s_original/'%date #### path of unsorted dicom files
trg_root = './0_0_dcm_glioma_original/a_%s/'%date  ### path to sort out dicom files

func_sort_dicoms(src_root, trg_root)

KeyboardInterrupt: 

#### Make a table of unitno / study date / sequence / fileno

In [ ]:
import pandas as pd
import string
import re


def func_make_dirtable(src_root):

    df_fileno = pd.DataFrame()
    df_fileno_sub = pd.DataFrame()

    for root, folders, files in os.walk(src_root):
        for folder in folders: 
            
            if ((len(folder) == 6 or len(folder) == 7 or len(folder) == 8) 
                and all(c in string.digits for c in folder)): ## if folde name is 6 or 7 or 8 digit numbers
                print(folder)
                unitno_subfolder = os.path.join(src_root, folder)
                unitno = folder
                for root2, dirs2, files2 in os.walk(unitno_subfolder):
                    if not dirs2:  # go to lowest subfolder with dicom files = sequence folder
                        sequence = os.path.basename(root2)
                        study_subfolder = os.path.dirname(root2)
                        study = os.path.basename(study_subfolder)
                        
                        # extract study name from study
                        r = re.compile("([0-9_]+)([a-zA-Z_]+)")
                        m = r.match(study)   # ex: 8751259_20181023_MRI_Brain_Intra_axial_Tumor_Preop__Diff_contrast_
                        if m :
                            study_only = m.group(2)  # ex: MRI_Brain_Intra_axial_Tumor_Preop__Diff_contrast_
                        
                        fileno = len(os.listdir(root2))
                        df_fileno_sub['unitno']=pd.Series(unitno).values
                        df_fileno_sub['study']=pd.Series(study).values
                        df_fileno_sub['study_only']=pd.Series(study_only).values
                        df_fileno_sub['sequence']=pd.Series(sequence).values
                        df_fileno_sub['number of files']=pd.Series(fileno).values
                        df_fileno_sub['seq2']=''
                        print(df_fileno_sub)
                        df_fileno = pd.concat([df_fileno, df_fileno_sub], axis=0)

    return df_fileno

In [ ]:
src_root = './0_0_dcm_glioma_original/a_%s/'%date
df_fileno = func_make_dirtable(src_root)

In [ ]:
df_fileno.to_csv('./glioma_original_dicom_unitno_study_seq_fileno_list_%s.csv'%date)

###  ToDo : Mark sequencenames.  0 - T1pre, 1 - T1gd, 2 - T2, 3 - FLAIR

###### call df_fileno with seq2 classified

In [8]:
total_list=pd.read_csv('glioma_original_dicom_unitno_study_seq_fileno_list_%s_seq2.csv'%date)
ptlist = pd.Series(total_list.iloc[:,1]).unique()

###### count number of sequences  : 0- t1pre, 1-t1gd, 2-t2, 3-flairGD, 4- flairpre

In [ ]:
seq_names = {0:'T1pre', 1: 'T1gd', 2: 'T2', 3: 'FLAIRgd', 4:'FLAIRpre'}

for unitno in ptlist:
    subdata = total_list.loc[total_list['unitno']==unitno]
    seq2_list = (subdata.iloc[:,-1])  #sequence were recorded in the last column
    seq2_list = [int(seq2) if not np.isnan(seq2) else seq2 for seq2 in seq2_list ]
    n_seq_list = []
    for i in [0,1,2,3]: # FLAIR pre is optional
        n_seq_list.append(seq2_list.count(i))
        if seq2_list.count(i) != 1:
            print('%s ---- %s # is %s'% (unitno, seq_names[i], seq2_list.count(i)))
            print(subdata['sequence'])
       


#### check correctness of image sequences

In [9]:
total_list=pd.read_csv('glioma_original_dicom_unitno_study_seq_fileno_list_%s_seq2.csv'%date)
ptlist = pd.Series(total_list.iloc[:,1]).unique()
sorted_dir = './0_0_dcm_glioma_original/a_%s/'%date


In [14]:

def func_imshow_midslice(unitno, total_list, src_root):
    seq_names = {0:'T1pre', 1: 'T1gd', 2: 'T2', 3: 'FLAIRgd', 4:'FLAIRpre'}

    subdata = total_list.loc[total_list['unitno']==unitno]
    seq2_list = (subdata.iloc[:,-1])  #sequence were recorded in the last column
    seq2_list = [int(seq2) if not np.isnan(seq2) else seq2 for seq2 in seq2_list ]
    
    for seq_no in range(5):
        if not seq_no in seq2_list:
            print("%s__%s___No corresponding sequence"%(unitno, seq_names[seq_no]))
        elif seq2_list.count(seq_no) > 1:
            print("%s__%s___>=2 corresponding sequence"%(unitno, seq_names[seq_no]))

        elif seq2_list.count(seq_no) == 1: 
            study = subdata.loc[subdata['seq2']==seq_no, 'study'].item()
            seq = subdata.loc[subdata['seq2']==seq_no, 'sequence'].item()
            folder_path = os.path.join(src_root, str(int(unitno)), study, seq)
            #fileno = len([f for f in os.listdir(folder_path) if f.endswith('.dcm')])
            #mid_z = int(fileno/2)
            mid_z = 12

            img_file = os.path.join(folder_path, os.listdir(folder_path)[mid_z])
            img = pydicom.dcmread(img_file)
            plt.imshow(img.pixel_array, cmap='gray')
            plt.title("%s___%s"%(unitno, seq_names[seq_no]))
            plt.show()

In [16]:
import time
from IPython.display import clear_output


for i in range(41, len(ptlist)):
    print(i)
    unitno = ptlist[i]
    func_imshow_midslice(unitno=unitno, total_list=total_list, src_root=sorted_dir)
    time.sleep(4)
    clear_output()

###  pick T1pre / T1gd / T2 / FLAIR according to csv table -> convert to Nifti 

In [17]:
import os
import pandas as pd
import shutil
import string

total_list=pd.read_csv('glioma_original_dicom_unitno_study_seq_fileno_list_%s_seq2.csv'%date)
ptlist = pd.Series(total_list.iloc[:,1]).unique()



In [18]:
src_root = './0_0_dcm_glioma_original/a_%s/'%date

trg_roots = {0:'./0_5_nifti_glioma_original/t1pre/', 1: './0_5_nifti_glioma_original/t1gd/', 
             2: './0_5_nifti_glioma_original/t2/', 3: './0_5_nifti_glioma_original/flair/',
            4: './0_5_nifti_glioma_original/flair_pre/'}
seq_names = {0:'t1pre', 1: 't1gd', 2: 't2', 3: 'flair', 4:'flair_pre'}



In [29]:
from nipype.interfaces.dcm2nii import Dcm2nii
from nipype.interfaces.fsl import ChangeDataType

def func_sort_to_nifti(unitno, total_list, src_root, trg_roots, seq_names):    
    subdata = total_list.loc[total_list['unitno']==unitno]
    for j in range(len(subdata)):
        study = str(subdata['study'].iloc[j])
        seq = str(subdata['sequence'].iloc[j])
        seq2 = subdata['seq2'].iloc[j]  #sequence were recorded in the last column
        
        if not np.isnan(seq2):
            seq2 = int(seq2)      
        
        if seq2 in [0, 1, 2, 3]:                
            newfile = "%s_%s_original.nii.gz" % (str(int(unitno)), seq_names[seq2])
            newfile2 = "%s_%s_original2.nii.gz" % (str(int(unitno)), seq_names[seq2])
            new_folder = trg_roots[seq2]
            
            src_folder = os.path.join(src_root, str(int(unitno)), study, seq)
            
            if not os.path.isfile(os.path.join(new_folder, newfile2)):
                converter = Dcm2nii(source_dir = src_folder,
                                    gzip_output = True,
                                    output_dir = new_folder)
                converter.run()

                os.rename(converter.output_files[0],
                         os.path.join(new_folder, newfile))

                ## make sure datatype == float-32bit
                img_nifti = sitk.ReadImage(os.path.join(new_folder, newfile))
                img_type = img_nifti.GetPixelIDTypeAsString()
                print(img_type)
                if img_type == '32-bit float' :
                    os.rename(os.path.join(new_folder, newfile),
                              os.path.join(new_folder, newfile2))

                else:
                    print('*************************' + newfile)
                    change_type = ChangeDataType(in_file = os.path.join(new_folder, newfile),
                                                 output_datatype = 'float',
                                                 out_file = os.path.join(new_folder, newfile2),
                                                 output_type = 'NIFTI_GZ')
                    change_type.run()





In [ ]:
for unitno in ptlist:
    print(unitno)
    func_sort_to_nifti(unitno, total_list, src_root, trg_roots, seq_names)

10000818
10005178
190804-23:58:17,93 interface INFO:
	 stdout 2019-08-04T23:58:17.093523:Chris Rorden's dcm2nii :: 2MAY2016 64bit BSD License  :: (upgrade to dcm2niix suggested)
190804-23:58:17,96 interface INFO:
	 stdout 2019-08-04T23:58:17.093523:reading preferences file config.ini
190804-23:58:17,98 interface INFO:
	 stdout 2019-08-04T23:58:17.093523:Data will be exported to ./0_5_nifti_glioma_original/t2/
190804-23:58:17,100 interface INFO:
	 stdout 2019-08-04T23:58:17.093523:Validating 22 potential DICOM images.
190804-23:58:17,101 interface INFO:
	 stdout 2019-08-04T23:58:17.093523:Found 22 DICOM images.
190804-23:58:18,113 interface INFO:
	 stdout 2019-08-04T23:58:18.113881:Converting 22/22  volumes: 1
190804-23:58:18,115 interface INFO:
	 stdout 2019-08-04T23:58:18.113881:10005178_20190507_501_1.dcm->20190507_215525SyntheticMRSEs501a1000.nii
190804-23:58:18,117 interface INFO:
	 stdout 2019-08-04T23:58:18.113881: brightest voxel was 9312: data will be saved as 16-bit signed int

190805-00:00:30,316 interface INFO:
	 stdout 2019-08-05T00:00:30.310283:Found 20 DICOM images.
190805-00:00:31,329 interface INFO:
	 stdout 2019-08-05T00:00:31.329361:Converting 20/20  volumes: 1
190805-00:00:31,330 interface INFO:
	 stdout 2019-08-05T00:00:31.329361:10012748_20190604_601_1.dcm->20190604_221622WIPT1AXs601a1006.nii
190805-00:00:31,332 interface INFO:
	 stdout 2019-08-05T00:00:31.329361: brightest voxel was 2229: data will be saved as 16-bit signed integer.
190805-00:00:31,333 interface INFO:
	 stdout 2019-08-05T00:00:31.329361:GZip...20190604_221622WIPT1AXs601a1006.nii.gz
32-bit float
190805-00:00:32,589 interface INFO:
	 stdout 2019-08-05T00:00:32.589655:Chris Rorden's dcm2nii :: 2MAY2016 64bit BSD License  :: (upgrade to dcm2niix suggested)
190805-00:00:32,591 interface INFO:
	 stdout 2019-08-05T00:00:32.589655:reading preferences file config.ini
190805-00:00:32,592 interface INFO:
	 stdout 2019-08-05T00:00:32.589655:Data will be exported to ./0_5_nifti_glioma_origina

190805-00:01:37,590 interface INFO:
	 stdout 2019-08-05T00:01:37.590800:Converting 22/22  volumes: 1
190805-00:01:37,592 interface INFO:
	 stdout 2019-08-05T00:01:37.590800:1974254_20180826_601_1.dcm->20180826_020102WIPT1AXs601a1006.nii
190805-00:01:37,593 interface INFO:
	 stdout 2019-08-05T00:01:37.590800: brightest voxel was 2077: data will be saved as 16-bit signed integer.
190805-00:01:37,594 interface INFO:
	 stdout 2019-08-05T00:01:37.590800:GZip...20180826_020102WIPT1AXs601a1006.nii.gz
32-bit float
190805-00:01:38,866 interface INFO:
	 stdout 2019-08-05T00:01:38.866062:Chris Rorden's dcm2nii :: 2MAY2016 64bit BSD License  :: (upgrade to dcm2niix suggested)
190805-00:01:38,867 interface INFO:
	 stdout 2019-08-05T00:01:38.866062:reading preferences file config.ini
190805-00:01:38,869 interface INFO:
	 stdout 2019-08-05T00:01:38.866062:Data will be exported to ./0_5_nifti_glioma_original/flair/
190805-00:01:38,870 interface INFO:
	 stdout 2019-08-05T00:01:38.866062:Validating 22 p

190805-00:02:43,395 interface INFO:
	 stdout 2019-08-05T00:02:43.392587: brightest voxel was 814: data will be saved as 16-bit signed integer.
190805-00:02:43,397 interface INFO:
	 stdout 2019-08-05T00:02:43.392587:GZip...20190222_2336173DT1TFEAXFGDs1302a1013.nii.gz
190805-00:02:43,398 interface INFO:
	 stdout 2019-08-05T00:02:43.392587:This image will not be reoriented (larger than MaxReorientMatrix= 1023
32-bit float
190805-00:02:47,780 interface INFO:
	 stdout 2019-08-05T00:02:47.780729:Chris Rorden's dcm2nii :: 2MAY2016 64bit BSD License  :: (upgrade to dcm2niix suggested)
190805-00:02:47,782 interface INFO:
	 stdout 2019-08-05T00:02:47.780729:reading preferences file config.ini
190805-00:02:47,783 interface INFO:
	 stdout 2019-08-05T00:02:47.780729:Data will be exported to ./0_5_nifti_glioma_original/t1pre/
190805-00:02:47,784 interface INFO:
	 stdout 2019-08-05T00:02:47.780729:Validating 20 potential DICOM images.
190805-00:02:47,785 interface INFO:
	 stdout 2019-08-05T00:02:47.7

190805-00:03:32,232 interface INFO:
	 stdout 2019-08-05T00:03:32.228457:GZip...20180613_095724T2AXs501a1005.nii.gz
32-bit float
2826490
190805-00:03:34,462 interface INFO:
	 stdout 2019-08-05T00:03:34.462040:Chris Rorden's dcm2nii :: 2MAY2016 64bit BSD License  :: (upgrade to dcm2niix suggested)
190805-00:03:34,464 interface INFO:
	 stdout 2019-08-05T00:03:34.462040:reading preferences file config.ini
190805-00:03:34,465 interface INFO:
	 stdout 2019-08-05T00:03:34.462040:Data will be exported to ./0_5_nifti_glioma_original/t1gd/
190805-00:03:34,467 interface INFO:
	 stdout 2019-08-05T00:03:34.462040:Validating 170 potential DICOM images.
190805-00:03:34,468 interface INFO:
	 stdout 2019-08-05T00:03:34.462040:Found 170 DICOM images.
190805-00:03:56,930 interface INFO:
	 stdout 2019-08-05T00:03:56.930286:Converting 170/170  volumes: 1
190805-00:03:56,931 interface INFO:
	 stdout 2019-08-05T00:03:56.930286:2826490_20180728_1102_1.dcm->20180728_2036043DT1TFEAXFGDs1102a1011.nii
190805-00:0

...

190805-00:05:48,520 interface INFO:
	 stdout 2019-08-05T00:05:48.517929:reading preferences file config.ini
190805-00:05:48,521 interface INFO:
	 stdout 2019-08-05T00:05:48.517929:Data will be exported to ./0_5_nifti_glioma_original/flair/
190805-00:05:48,522 interface INFO:
	 stdout 2019-08-05T00:05:48.517929:Validating 21 potential DICOM images.
190805-00:05:48,523 interface INFO:
	 stdout 2019-08-05T00:05:48.517929:Found 21 DICOM images.
190805-00:05:50,47 interface INFO:
	 stdout 2019-08-05T00:05:50.047067:Converting 21/21  volumes: 1
190805-00:05:50,48 interface INFO:
	 stdout 2019-08-05T00:05:50.047067:3720122_20190412_1401_1.dcm->20190412_034516WIPT2FLAIRGDs1401a1014.nii
190805-00:05:50,50 interface INFO:
	 stdout 2019-08-05T00:05:50.047067: brightest voxel was 2552: data will be saved as 16-bit signed integer.
190805-00:05:50,51 interface INFO:
	 stdout 2019-08-05T00:05:50.047067:GZip...20190412_034516WIPT2FLAIRGDs1401a1014.nii.gz
32-bit float
3876719
190805-00:05:52,275 interf

190805-00:07:04,982 interface INFO:
	 stdout 2019-08-05T00:07:04.978684:Data will be exported to ./0_5_nifti_glioma_original/flair/
190805-00:07:04,984 interface INFO:
	 stdout 2019-08-05T00:07:04.978684:Validating 22 potential DICOM images.
190805-00:07:04,985 interface INFO:
	 stdout 2019-08-05T00:07:04.978684:Found 22 DICOM images.
190805-00:07:05,997 interface INFO:
	 stdout 2019-08-05T00:07:05.997683:Converting 22/22  volumes: 1
190805-00:07:05,999 interface INFO:
	 stdout 2019-08-05T00:07:05.997683:4199579_20180630_1201_1.dcm->20180630_205202WIPT2FLAIRGDs1201a1012.nii
190805-00:07:06,0 interface INFO:
	 stdout 2019-08-05T00:07:05.997683: brightest voxel was 1429: data will be saved as 16-bit signed integer.
190805-00:07:06,1 interface INFO:
	 stdout 2019-08-05T00:07:05.997683:GZip...20180630_205202WIPT2FLAIRGDs1201a1012.nii.gz
32-bit float
4250762
190805-00:07:08,755 interface INFO:
	 stdout 2019-08-05T00:07:08.755094:Chris Rorden's dcm2nii :: 2MAY2016 64bit BSD License  :: (upgr

190805-00:08:48,218 interface INFO:
	 stdout 2019-08-05T00:08:48.216448:reading preferences file config.ini
190805-00:08:48,220 interface INFO:
	 stdout 2019-08-05T00:08:48.216448:Data will be exported to ./0_5_nifti_glioma_original/t2/
190805-00:08:48,221 interface INFO:
	 stdout 2019-08-05T00:08:48.216448:Validating 20 potential DICOM images.
190805-00:08:48,222 interface INFO:
	 stdout 2019-08-05T00:08:48.216448:Found 20 DICOM images.
190805-00:08:49,235 interface INFO:
	 stdout 2019-08-05T00:08:49.235353:Converting 20/20  volumes: 1
190805-00:08:49,236 interface INFO:
	 stdout 2019-08-05T00:08:49.235353:5005671_20181108_701_1.dcm->20181108_002818T2AXs701a1007.nii
190805-00:08:49,238 interface INFO:
	 stdout 2019-08-05T00:08:49.235353: brightest voxel was 2036: data will be saved as 16-bit signed integer.
190805-00:08:49,239 interface INFO:
	 stdout 2019-08-05T00:08:49.235353:GZip...20181108_002818T2AXs701a1007.nii.gz
32-bit float
190805-00:08:50,444 interface INFO:
	 stdout 2019-08

190805-00:10:27,100 interface INFO:
	 stdout 2019-08-05T00:10:27.097566:reading preferences file config.ini
190805-00:10:27,101 interface INFO:
	 stdout 2019-08-05T00:10:27.097566:Data will be exported to ./0_5_nifti_glioma_original/t1pre/
190805-00:10:27,102 interface INFO:
	 stdout 2019-08-05T00:10:27.097566:Validating 21 potential DICOM images.
190805-00:10:27,104 interface INFO:
	 stdout 2019-08-05T00:10:27.097566:Found 21 DICOM images.
190805-00:10:28,117 interface INFO:
	 stdout 2019-08-05T00:10:28.117589:Converting 21/21  volumes: 1
190805-00:10:28,118 interface INFO:
	 stdout 2019-08-05T00:10:28.117589:5240823_20180626_601_1.dcm->20180626_034546WIPT1AXs601a1006.nii
190805-00:10:28,120 interface INFO:
	 stdout 2019-08-05T00:10:28.117589: brightest voxel was 1866: data will be saved as 16-bit signed integer.
190805-00:10:28,121 interface INFO:
	 stdout 2019-08-05T00:10:28.117589:GZip...20180626_034546WIPT1AXs601a1006.nii.gz
32-bit float
190805-00:10:29,390 interface INFO:
	 stdou

190805-00:11:12,6 interface INFO:
	 stdout 2019-08-05T00:11:12.001346:Validating 170 potential DICOM images.
190805-00:11:12,8 interface INFO:
	 stdout 2019-08-05T00:11:12.001346:Found 170 DICOM images.
190805-00:11:38,45 interface INFO:
	 stdout 2019-08-05T00:11:38.044936:Converting 170/170  volumes: 1
190805-00:11:38,46 interface INFO:
	 stdout 2019-08-05T00:11:38.044936:5324864_20181013_1202_1.dcm->20181013_1806473DT1TFEAXFGDs1202a1012.nii
190805-00:11:38,48 interface INFO:
	 stdout 2019-08-05T00:11:38.044936: brightest voxel was 1026: data will be saved as 16-bit signed integer.
190805-00:11:38,49 interface INFO:
	 stdout 2019-08-05T00:11:38.044936:GZip...20181013_1806473DT1TFEAXFGDs1202a1012.nii.gz
190805-00:11:38,50 interface INFO:
	 stdout 2019-08-05T00:11:38.044936:This image will not be reoriented (larger than MaxReorientMatrix= 1023
32-bit float
190805-00:11:42,602 interface INFO:
	 stdout 2019-08-05T00:11:42.602338:Chris Rorden's dcm2nii :: 2MAY2016 64bit BSD License  :: (up

#### TODO : check nifti orientation and quality